<a href="https://colab.research.google.com/github/WestonMadeira1/HW_6/blob/main/HW6_P1A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from google.colab import drive
drive.mount('/content/drive')

file_path = '/content/drive/My Drive/ML Data/Housing.csv'
housing_data = pd.read_csv(file_path)

varlist =  ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea','furnishingstatus']

def binary_map(x):
    return x.map({'yes': 1, 'furnished':1, 'semi-furnished':0.5, 'unfurnished':0, 'no': 0})

housing_data[varlist] = housing_data[varlist].apply(binary_map)
housing_data.head()
target_column = 'price'


X = housing_data.drop(target_column, axis=1).values
y = housing_data[target_column].values.reshape(-1, 1)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the input features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Convert numpy arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val_scaled, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

# Create PyTorch DataLoader for training and validation sets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

def training_loop(n_epochs, optimizer, model, loss_fn, t_u_train, t_u_val, t_c_train,
                  t_c_val):
  for epoch in range (1, n_epochs + 1) :
    t_p_train = model (t_u_train)
    loss_train = loss_fn(t_p_train, t_c_train)

    t_p_val = model (t_u_val)

    loss_val = loss_fn(t_p_val, t_c_val)
    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()

    if epoch == 1 or epoch % 1000 == 0:
      print(f"Epoch {epoch}, Training Loss {loss_train.item(): .4f}",
            f"Validation Loss {loss_val.item():.4f}")


# Define the neural network model
seq_model = nn.Sequential(
            nn.Linear(12,36),
            nn.ReLU(),
            nn.Linear(36,1))
seq_model


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Sequential(
  (0): Linear(in_features=12, out_features=36, bias=True)
  (1): ReLU()
  (2): Linear(in_features=36, out_features=1, bias=True)
)

In [ ]:
[param.shape for param in seq_model.parameters()]

[torch.Size([36, 12]), torch.Size([36]), torch.Size([1, 36]), torch.Size([1])]

In [ ]:
for name, param in seq_model.named_parameters():
  print(name, param.shape)

0.weight torch.Size([36, 12])
0.bias torch.Size([36])
2.weight torch.Size([1, 36])
2.bias torch.Size([1])


In [ ]:
from torch.optim import Adam
optimizer = optim.Adam(seq_model.parameters(), lr=0.001)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer,
    model = seq_model,
    loss_fn=nn.MSELoss(),
    t_u_train = X_train_tensor,
    t_u_val = X_val_tensor,
    t_c_train = y_train_tensor,
    t_c_val = y_val_tensor)

#print('output', seq_model(X_val_tensor))
#print('answer', y_val_tensor)
#print('hidden', seq_model.0.weight.grad)

Epoch 1, Training Loss  25234788843520.0000 Validation Loss 30129992499200.0000
Epoch 1000, Training Loss  25232075128832.0000 Validation Loss 30126752399360.0000
Epoch 2000, Training Loss  25224483438592.0000 Validation Loss 30117654953984.0000
Epoch 3000, Training Loss  25213246898176.0000 Validation Loss 30104218501120.0000
Epoch 4000, Training Loss  25198956904448.0000 Validation Loss 30087149780992.0000
Epoch 5000, Training Loss  25181902864384.0000 Validation Loss 30066778046464.0000
